In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


In [2]:
"""
1.Basic Embedding Model
    1-1. NNLM(Neural Network Language Model)
"""

dtype = torch.FloatTensor
sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()  # 製作詞彙表
print(word_list)
word_list = list(set(word_list))  # 去除詞彙表中的重複元素
print("去重後的word_list:", word_list)
word_dict = {w: i for i, w in enumerate(word_list)}  # 將每個單詞對應於相應的索引
number_dict = {i: w for i, w in enumerate(word_list)}  # 將每個索引對應相應的單詞
n_class = len(word_dict)  # 單詞總數

# NNLM parameters
n_step = 2   # 根據前兩個單詞預測第3個單詞
n_hidden = 2  # 隱藏層神經元個數
m = 2  # 詞向量的維度

['i', 'like', 'dog', 'i', 'love', 'coffee', 'i', 'hate', 'milk']
去重後的word_list: ['like', 'milk', 'i', 'love', 'hate', 'coffee', 'dog']


In [3]:
# 將原始資料用batch餵給模型

def make_batch(sentences):
    input_batch = []
    target_batch = []
    for sentence in sentences:
        word = sentence.split()
        input = [word_dict[w] for w in word[:-1]]
        target = word_dict[word[-1]]
        input_batch.append(input)
        target_batch.append(target)
    return input_batch, target_batch

In [4]:
# Model

class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, embedding_dim=m)
        self.H = nn.Parameter(torch.randn(n_step * m, n_hidden).type(dtype))
        self.W = nn.Parameter(torch.randn(n_step * m, n_class).type(dtype))
        self.d = nn.Parameter(torch.randn(n_hidden).type(dtype))
        self.U = nn.Parameter(torch.randn(n_hidden, n_class).type(dtype))
        self.b = nn.Parameter(torch.randn(n_class).type(dtype))

    def forward(self, x):
        x = self.C(x)
        x = x.view(-1, n_step * m)
        # x: [batch_size, n_step*n_class]
        tanh = torch.tanh(self.d + torch.mm(x, self.H))
        # tanh: [batch_size, n_hidden]
        output = self.b + torch.mm(x, self.W) + torch.mm(tanh, self.U)
        # output: [batch_size, n_class]
        return output


model = NNLM()

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 製作輸入
input_batch, target_batch = make_batch(sentences)
input_batch = Variable(torch.LongTensor(input_batch))
target_batch = Variable(torch.LongTensor(target_batch))

In [6]:
# 開始訓練
for epoch in range(5000):
    optimizer.zero_grad()
    output = model(input_batch)
# output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print("Epoch:{}".format(epoch + 1), "Loss:{:.3f}".format(loss))
    loss.backward()
    optimizer.step()

Epoch:1000 Loss:0.149
Epoch:2000 Loss:0.018
Epoch:3000 Loss:0.006
Epoch:4000 Loss:0.003
Epoch:5000 Loss:0.001


In [7]:
# 預測
predict = model(input_batch).data.max(
    1, keepdim=True)[1]  # [batch_size, n_class]
print("predict: \n", predict)
# 測試
print([sentence.split()[:2] for sentence in sentences], "---->",
      [number_dict[n.item()] for n in predict.squeeze()])


predict: 
 tensor([[6],
        [5],
        [1]])
[['i', 'like'], ['i', 'love'], ['i', 'hate']] ----> ['dog', 'coffee', 'milk']
